## Load required libraries

In [52]:
from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel
from gensim.models.ldamulticore import LdaMulticore
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
import numpy as np
import pandas as pd
from tqdm import tqdm
import nltk
import re
import gensim, spacy
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis 
import altair as alt
from nltk.corpus import stopwords
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
pyLDAvis.enable_notebook()

## Required Utilities 

### Pre-processing Utilities 

In [54]:
allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use', 'not', 'would', 'say', 'could', '_', 'be', 'know', 'good', 'go', 'get', 'do', 'done', 'try', \
                   'many', 'some', 'nice', 'thank', 'think', 'see', 'rather', 'easy', 'easily', 'lot', 'lack', 'make', 'want', 'seem', 'run', 'need',  \
                   'even', 'right', 'line', 'even', 'also', 'may', 'take', 'come'])

def utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=None):
    ## clean (convert to lowercase and remove punctuations and characters and then strip)
    lst_text = text.split()
    ## choose only alphbetical words and filter words less than 3 chars
#     lst_text = [token.lower() for token in lst_text if token.isalpha() and len(token) >=3]      
    text = re.sub(r'[^\w\s]', '', str(text).lower().strip())
            
    ## Tokenize (convert from string to list)
#     lst_text = text.split()
    ## remove Stopwords
    if lst_stopwords is not None:
        lst_text = [word for word in lst_text if word not in 
                    lst_stopwords]
                
    ## Stemming (remove -ing, -ly, ...)
    if flg_stemm == True:
        ps = nltk.stem.porter.PorterStemmer()
        lst_text = [ps.stem(word) for word in lst_text]
                
    ## Lemmatisation (convert the word into root word)
    if flg_lemm == True:
        lem = nltk.stem.wordnet.WordNetLemmatizer()
        lst_text = [lem.lemmatize(word) for word in lst_text]
    
    lst_text = [token for token in lst_text if len(token) >=3]
            
    ## back to string from list
#     text = " ".join(lst_text)
    return lst_text


def ngram_processing(texts):
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(texts, min_count=5, threshold=100)
    trigram = gensim.models.Phrases(bigram[texts], threshold=100)

    # Fastest way to get a sentence clubbed as trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)   
    
    bigram_words = [bigram_mod[doc] for doc in texts]
    trigram_words = [trigram_mod[bigram_mod[doc]] for doc in bigram_words]
    return(trigram_words)


def lemmatization(texts, allowed_postags=allowed_postags):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return(texts_out)

### Model Creation Utilities 

In [15]:
def model_creation(corpus, id2word, num_topics, workers=None):
    if not workers:
        lda = LdaModel(corpus, id2word = id2word, num_topics=num_topics,
                              random_state=696, update_every=1,
                              chunksize=10, passes=10, 
                              alpha='symmetric', iterations=100,
                              per_word_topics=True)
    else:
        lda = LdaMulticore(corpus, id2word = id2word, workers = workers, num_topics=num_topics,
                              random_state=696,
                              chunksize=10, passes=10, 
                              alpha='symmetric', iterations=100,
                              per_word_topics=True)
        
    
    return(lda)

def compute_coherence_values(id2word, corpus, texts, workers, limit, start=2, step=2):
    coherence_values = []
    perplexity_values = []
    model_list = []
    num_topics_list = []
    
    for num_topics in tqdm(range(start, limit, step)):
        lda_model = model_creation(corpus, id2word, num_topics, workers)
        perplexity_score = lda_model.log_perplexity(corpus)

        coherence_model = CoherenceModel(model=lda_model, texts=texts,
                                dictionary=id2word,coherence='c_v')
        coherence_score = coherence_model.get_coherence()
        model_list.append(lda_model)
        coherence_values.append(coherence_score)
        perplexity_values.append(perplexity_score)
        num_topics_list.append(num_topics)
    return(num_topics_list, model_list, coherence_values, perplexity_values)
        

## Load Data

In [5]:
# Read articles for 2004 for test purpose
basepath = '/mnt/d/Amit/data-science/MADS/SIADS696/'

filepath = 'datasets/raw_data/gzip_data/'
filename = 'articles_2004.jl.gz'
filepath = basepath + filepath + filename
filepath

'/mnt/d/Amit/data-science/MADS/SIADS696/datasets/raw_data/gzip_data/articles_2004.jl.gz'

In [7]:
articles_df = pd.read_json(filepath, lines=True)
articles_df = articles_df.dropna()
articles_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 53394 entries, 0 to 54985
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   id        53394 non-null  int64         
 1   date      53394 non-null  datetime64[ns]
 2   url       53394 non-null  object        
 3   title     53394 non-null  object        
 4   category  53394 non-null  object        
 5   article   53394 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 2.9+ MB


### Pre-processing Text

In [32]:
%%time
articles_df['clean_text_test'] = articles_df['article'].apply(lambda text : 
                        utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=stop_words))

articles_df.head()

CPU times: user 51.4 s, sys: 0 ns, total: 51.4 s
Wall time: 51.4 s


,id,date,url,title,category,article,clean_text,clean_text_test
0,1,2004-01-01,https://timesofindia.indiatimes.com/business/i...,economy-breaks-8-pc-barrier,"[business, india-business]",NEW DELHI: The feel-good factor got a boost on...,"[new, factor, got, boost, last, day, indian, e...","[NEW, DELHI:, The, feel-good, factor, got, boo..."
1,2,2004-01-01,https://timesofindia.indiatimes.com/world/paki...,jaish-leader-missing-after-attacks,"[world, pakistan]",ISLAMABAD: Some activists of the banned milita...,"[activist, banned, militant, outfit, arrested,...","[ISLAMABAD:, Some, activist, banned, militant,..."
2,3,2004-01-01,https://timesofindia.indiatimes.com/world/us/1...,1m-for-jackson-interview-to-cbs,"[world, us]","&lt;div class=""section1""&gt;&lt;div class=""Nor...","[michael, jackson, struck, deal, cbs, paid, ef...","[&lt;div, class=""section1""&gt;&lt;div, class=""..."
3,4,2004-01-01,https://timesofindia.indiatimes.com/india/sars...,sars-screening-takes-off-at-igi,"[timesofindia.indiatimes.com, india]",NEW DELHI: With the SARS virus raising its hea...,"[new, sars, virus, raising, head, airport, aut...","[NEW, DELHI:, With, SARS, virus, raising, head..."
4,5,2004-01-01,https://timesofindia.indiatimes.com/india/indi...,india-to-test-fire-agni-iii-soon,"[timesofindia.indiatimes.com, india]",NEW DELHI: The nuclear-capable â€˜Agni-IIIâ€™ ...,"[new, capable, hitting, strategic, target, dee...","[NEW, DELHI:, The, nuclear-capable, â€˜Agni-II..."


In [8]:
%%time
articles_df['clean_text'] = articles_df['article'].apply(lambda text : 
                        utils_preprocess_text(text, flg_stemm=False, flg_lemm=True, lst_stopwords=stop_words))

articles_df.head()

CPU times: user 36.5 s, sys: 0 ns, total: 36.5 s
Wall time: 36.5 s


,id,date,url,title,category,article,clean_text
0,1,2004-01-01,https://timesofindia.indiatimes.com/business/i...,economy-breaks-8-pc-barrier,"[business, india-business]",NEW DELHI: The feel-good factor got a boost on...,"[new, factor, got, boost, last, day, indian, e..."
1,2,2004-01-01,https://timesofindia.indiatimes.com/world/paki...,jaish-leader-missing-after-attacks,"[world, pakistan]",ISLAMABAD: Some activists of the banned milita...,"[activist, banned, militant, outfit, arrested,..."
2,3,2004-01-01,https://timesofindia.indiatimes.com/world/us/1...,1m-for-jackson-interview-to-cbs,"[world, us]","&lt;div class=""section1""&gt;&lt;div class=""Nor...","[michael, jackson, struck, deal, cbs, paid, ef..."
3,4,2004-01-01,https://timesofindia.indiatimes.com/india/sars...,sars-screening-takes-off-at-igi,"[timesofindia.indiatimes.com, india]",NEW DELHI: With the SARS virus raising its hea...,"[new, sars, virus, raising, head, airport, aut..."
4,5,2004-01-01,https://timesofindia.indiatimes.com/india/indi...,india-to-test-fire-agni-iii-soon,"[timesofindia.indiatimes.com, india]",NEW DELHI: The nuclear-capable â€˜Agni-IIIâ€™ ...,"[new, capable, hitting, strategic, target, dee..."


In [16]:
# choosing number of articles to process for stub testing
n = 10000
texts = list(articles_df['clean_text'][:n].values)
text_grams = ngram_processing(texts)
data_lemmatized = lemmatization(text_grams, allowed_postags)

In [17]:
%%time
# model creation
texts = data_lemmatized
id2word = Dictionary(texts)
corpus = [id2word.doc2bow(doc) for doc in texts]

CPU times: user 1.31 s, sys: 29.6 ms, total: 1.34 s
Wall time: 1.34 s


In [196]:
%%time
# model creation
# time estimation with LDA single core
# num_topics_list, model_list, coherence_vals, perplexity_vals = compute_coherence_values(id2word, corpus,   \
#                                                                 texts,workers=None, limit=30, start=2, step=3)

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs


In [19]:
%%time
# time estimation with multicore LDA
# model creation
num_topics_list, model_list, coherence_vals, perplexity_vals = compute_coherence_values(id2word, corpus,    \
                                                                texts, workers=5, limit=30, start=2, step=3)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [28:46<00:00, 172.64s/it]

CPU times: user 23min 33s, sys: 9min 48s, total: 33min 22s
Wall time: 28min 46s


In [20]:
model_df = pd.DataFrame(list(zip(num_topics_list, coherence_vals, perplexity_vals)), 
                        columns = ['num_topics', 'coherence_score', 'perplexity_score']) 

In [21]:
model_df

,num_topics,coherence_score,perplexity_score
0,2,0.329787,-8.483017
1,5,0.428724,-8.439057
2,8,0.416579,-8.693746
3,11,0.392127,-9.561194
4,14,0.380911,-11.831976
5,17,0.372935,-12.910598
6,20,0.378009,-13.672018
7,23,0.362770,-14.264419
8,26,0.367707,-15.004560
9,29,0.384859,-15.466451


## Visualize Topic Distribution

In [22]:
base_chart = alt.Chart(model_df)
line = base_chart.mark_line().encode(
            x=alt.X('num_topics:N', axis=alt.Axis(title='Number of Topics')),
            y=alt.Y('coherence_score', axis=alt.Axis(title='Coherence Score')),
            tooltip = ['num_topics', 'coherence_score']
)

point = base_chart.mark_point(color='orange', size=50).encode(
            x=alt.X('num_topics:N', axis=alt.Axis(title='Number of Topics')),
            y=alt.Y('coherence_score', axis=alt.Axis(title='Coherence Score')),
            tooltip = ['num_topics', 'coherence_score']
)    


(line + point
).properties(width=500, height=300, title='Number of Topics vs Coherence Score'
).configure_title(fontSize=25
).configure_axis(grid=False, domain=False, 
                 labelFontSize=15,titleFontSize=20)

alt.LayerChart(...)

In [27]:

optimal_model_cond = np.where(model_df['coherence_score'] == max(model_df['coherence_score'].values))
optimal_model = model_list[optimal_model_cond[0][0]]

In [28]:
# Number of Topics = 5
gensimvis.prepare(optimal_model, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.022716  0.123031       1        1  22.595318
2     -0.232281  0.033511       2        1  20.213296
4      0.168629  0.053586       3        1  19.881460
3      0.082766  0.029045       4        1  18.800271
1      0.003602 -0.239174       5        1  18.509656, topic_info=          Term         Freq        Total Category  logprob  loglift
90      police  6196.000000  6196.000000  Default  30.0000  30.0000
249   minister  5044.000000  5044.000000  Default  29.0000  29.0000
1024     party  3716.000000  3716.000000  Default  28.0000  28.0000
1010  congress  2924.000000  2924.000000  Default  27.0000  27.0000
643      state  6856.000000  6856.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
305       make   875.260937  3611.407815   Topic5  -5.4523   0.2695
850     decide   666.753215  1554.285191   Topic5  -5.7244   0.8405
217       take   785.351129  4182.374149   Topic5  -5.5607   0.0144
23        last   719.548172  3080.704409   Topic5  -5.6482   0.2326
24         new   694.205273  5143.926270   Topic5  -5.6840  -0.3159

[392 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
1284      4  0.995106        academic
277       2  0.998066          accuse
277       5  0.001919          accuse
1005      3  0.998158  administrative
1738      4  0.992781       admission
...     ...       ...             ...
44        4  0.338849            year
44        5  0.055377            year
483       1  0.972039           young
483       2  0.023310           young
483       4  0.004662           young

[802 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 5, 4, 2])

In [29]:
# Number of Topics = 8
gensimvis.prepare(model_list[2], corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.086428  0.056821       1        1  16.328535
7     -0.165100  0.006973       2        1  14.062813
6      0.129686  0.056741       3        1  13.347056
1      0.068615 -0.258519       4        1  12.675245
4     -0.012391 -0.107240       5        1  12.624060
2      0.134107  0.000072       6        1  12.538953
5     -0.228676  0.062146       7        1  12.126992
3      0.160187  0.183005       8        1   6.296347, topic_info=            Term         Freq        Total Category  logprob  loglift
90        police  6527.000000  6527.000000  Default  30.0000  30.0000
617   government  6883.000000  6883.000000  Default  29.0000  29.0000
1024       party  3412.000000  3412.000000  Default  28.0000  28.0000
249     minister  5065.000000  5065.000000  Default  27.0000  27.0000
1010    congress  2603.000000  2603.000000  Default  26.0000  26.0000
...          ...          ...          ...      ...      ...      ...
342       mumbai   279.875968   930.602124   Topic8  -5.5141   1.5637
147          pay   295.148403  1523.259510   Topic8  -5.4610   1.1241
340      january   304.177563  2460.540459   Topic8  -5.4309   0.6747
1308      direct   260.279202   927.219048   Topic8  -5.5867   1.4948
24           new   246.069248  5417.306014   Topic8  -5.6429  -0.3265

[573 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1284      2  0.996975  academic
277       3  0.893682    accuse
277       4  0.012961    accuse
277       6  0.079712    accuse
277       8  0.012961    accuse
...     ...       ...       ...
44        8  0.057620      year
483       1  0.855898     young
483       2  0.085164     young
483       6  0.053227     young
483       8  0.002129     young

[1319 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 8, 7, 2, 5, 3, 6, 4])

In [55]:
# Number of Topics = 11
gensimvis.prepare(model_list[3], corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.014399 -0.160101       1        1  14.551015
1     -0.127408 -0.107382       2        1  13.481657
9     -0.113521 -0.071434       3        1  12.383005
0     -0.085096  0.078647       4        1  11.459509
5     -0.042289  0.172413       5        1  10.596638
3     -0.076100  0.065582       6        1   9.808813
8     -0.015803 -0.036589       7        1   8.147453
7     -0.024742  0.245468       8        1   6.550412
4      0.059935 -0.104686       9        1   5.730060
6      0.035735 -0.107941      10        1   5.476557
10     0.403689  0.026024      11        1   1.814880, topic_info=               Term         Freq        Total Category  logprob  loglift
90           police  6998.000000  6998.000000  Default  30.0000  30.0000
1024          party  4172.000000  4172.000000  Default  29.0000  29.0000
249        minister  5380.000000  5380.000000  Default  28.0000  28.0000
617      government  7310.000000  7310.000000  Default  27.0000  27.0000
1021         leader  2833.000000  2833.000000  Default  26.0000  26.0000
...             ...          ...          ...      ...      ...      ...
1676  investigation   119.766063   558.880828  Topic11  -5.1190   2.4688
1267         monday   126.419609   746.832209  Topic11  -5.0649   2.2329
85         official   152.539960  3550.966290  Topic11  -4.8771   0.8616
964            side   108.191782   454.351826  Topic11  -5.2206   2.5742
340         january   120.413372  2722.685765  Topic11  -5.1136   0.8907

[770 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
1490      4  0.990763     abuse
1284      8  0.996316  academic
3401      1  0.021943  accident
3401      4  0.976464  accident
277       1  0.991594    accuse
...     ...       ...       ...
44       11  0.000284      year
2533      4  0.027607      zone
2533      7  0.966251      zone
418       4  0.013613       zoo
418       9  0.980149       zoo

[1996 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 10, 1, 6, 4, 9, 8, 5, 7, 11])

/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:10: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _nlv = LooseVersion(_np_version)
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p17 = _nlv < LooseVersion("1.17")
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:12: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p18 = _nlv < LooseVersion("1.18")
/home/amitjha/miniconda3/envs/mads/lib/pytho

/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:10: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _nlv = LooseVersion(_np_version)
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p17 = _nlv < LooseVersion("1.17")
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:12: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p18 = _nlv < LooseVersion("1.18")
/home/amitjha/miniconda3/envs/mads/lib/pytho

/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:10: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  _nlv = LooseVersion(_np_version)
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:11: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p17 = _nlv < LooseVersion("1.17")
/home/amitjha/miniconda3/envs/mads/lib/python3.8/site-packages/pandas/compat/numpy/__init__.py:12: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  np_version_under1p18 = _nlv < LooseVersion("1.18")
/home/amitjha/miniconda3/envs/mads/lib/pytho